In [1]:
# Step 1: Import necessary libraries
# --------------------------------------
# Import required libraries for document retrieval, reranking, and logging setup.
from sentence_transformers import CrossEncoder
import logging

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder
from financerag.tasks import FinDER

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)

/opt/miniconda3/envs/financerag/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Step 2: Initialize FinDER Task
# --------------------------
# In this baseline example, we are using the FinDER task, one of the seven available tasks in this project.
# If you want to use a different task, for example, 'OtherTask', you can change the task initialization as follows:
#
# Example:
# from financerag.tasks import OtherTask
# finder_task = OtherTask()
#
# For this baseline, we proceed with FinDER.
finder_task = FinDER()

Loading data manually from Linq-AI-Research/FinDER...
Processing documents and queries based on actual schema...
Loaded 5832 unique documents into Corpus.
Loaded 5703 queries.
Example Query [b33fcee7]: Delta in CBOE Data & Access Solutions rev from 2021-23.


In [4]:
# Step 3: Initialize DenseRetriever model
# -------------------------------------
# Initialize the retrieval model using SentenceTransformers. This model will be responsible
# for encoding both the queries and documents into embeddings.
#
# You can replace 'intfloat/e5-large-v2' with any other model supported by SentenceTransformers.
# For example: 'BAAI/bge-large-en-v1.5', 'Linq-AI-Research/Linq-Embed-Mistral', etc.
encoder_model = SentenceTransformerEncoder(
    model_name_or_path='intfloat/e5-large-v2',
    query_prompt='query: ',
    doc_prompt='passage: ',
)

retrieval_model = DenseRetrieval(
    model=encoder_model
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/e5-large-v2


In [6]:
# Step 4: Perform retrieval
# ---------------------
# Use the model to retrieve relevant documents for given queries.
retrieval_model = DenseRetrieval(
    model=encoder_model
)

retrieval_result = finder_task.retrieve(
    retriever=retrieval_model
)

# Print a portion of the retrieval results to verify the output.
print(f"Retrieved results for {len(retrieval_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in retrieval_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query

INFO:financerag.retrieval.dense:Encoding queries...
Batches: 100%|██████████| 90/90 [00:47<00:00,  1.91it/s]
INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...
Batches: 100%|██████████| 92/92 [13:46<00:00,  8.99s/it]


Retrieved results for 5703 queries. Here's an example of the top 5 documents for the first query:

Query ID: b33fcee7
  Document 1: Document ID = 1f477e45adf26ed090a3fbcbfe0c8e58, Score = 0.8312142491340637
  Document 2: Document ID = 7dc3a49ae571c589d4a4ddd5d73c4dd8, Score = 0.8190010786056519
  Document 3: Document ID = fdd9ce74769e2c2b49fa7b643f19d60d, Score = 0.818618893623352
  Document 4: Document ID = 0c411a2ad87adf98b9f9d2c7e556c113, Score = 0.8131804466247559
  Document 5: Document ID = c12efd2b6f71b9c479dfffecb71528bb, Score = 0.8096561431884766


In [7]:
# Step 5: Initialize CrossEncoder Reranker
# --------------------------------------
# The CrossEncoder model will be used to rerank the retrieved documents based on relevance.
#
# You can replace 'cross-encoder/ms-marco-MiniLM-L-12-v2' with any other model supported by CrossEncoder.
# For example: 'cross-encoder/ms-marco-TinyBERT-L-2', 'cross-encoder/stsb-roberta-large', etc.
reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
)

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps


In [8]:
# Step 6: Perform reranking
# -------------------------
# Rerank the top 100 retrieved documents using the CrossEncoder model.
reranking_result = finder_task.rerank(
    reranker=reranker,
    results=retrieval_result,
    top_k=100,  # Rerank the top 100 documents
    batch_size=32
)

# Print a portion of the reranking results to verify the output.
print(f"Reranking results for {len(reranking_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in reranking_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....
Batches:   0%|          | 38/17822 [00:42<5:32:38,  1.12s/it]


KeyboardInterrupt: 

In [ ]:
# Step 7: Save results
# -------------------
# Save the results to the specified output directory as a CSV file.
output_dir = './results'
finder_task.save_results(output_dir=output_dir)

# Confirm the results have been saved.
print(f"Results have been saved to {output_dir}/FinDER/results.csv")